In [43]:
import os
import requests
import random
import pickle
KEY = '91mTqXFEeSaCIbjC0fuaB1RABk4HANqS4I91qRBN'
KEY2 = 'flLQZGa0kx8opciUA6M7R9iLPwfFPj5yFKKlCiVb'

MOST_RECENT_HR_BILL = 6536 #1490
MOST_RECENT_S_BILL = 3548  #606
CONGRESS = 114

In [74]:
import re 
from lxml import html
temp = set()
page = requests.get('https://www.congress.gov/search?q={%22type%22:%22bills%22,%22source%22:%22legislation%22,%22congress%22:%22114%22,%22bill-status%22:%22passed-one%22,%22chamber%22:%22Senate%22}&pageSize=250&page=1')
tree = html.fromstring(page.content)
bills = tree.xpath('//span[@class="result-heading"]//a/text()')
temp |= set(bills)

senate_passed_one_house = set()
for bill in temp:
    senate_passed_one_house.add(int(''.join(list(filter(str.isdigit, bill)))))

    

temp = set()    
for i in range(1, 10):
    page = requests.get('https://www.congress.gov/search?pageSort=documentNumber:asc&q={%22congress%22:%22114%22,%22type%22:%22bills%22,%22bill-status%22:[%22passed-one%22],%22chamber%22:%22House%22}&pageSize=250&page=' + str(i))
    tree = html.fromstring(page.content)
    bills = tree.xpath('//span[@class="result-heading"]//a/text()')
    temp |= set(bills)
    
hr_passed_one_house = set()
for bill in temp:
    hr_passed_one_house.add(int(''.join(list(filter(str.isdigit, bill)))))


In [76]:
def getJSON(webpage):
    headers = {
        'X-API-Key': KEY
    }
    r = requests.get(webpage, headers=headers)
    return r.json()

def getSponsors(bill_id, congress, progress = 'introduced'):
    '''Returns (PrimarySponsor, [cosponsors]) for specified bill_id and congress strings. Returns () if faulty'''
    '''progress = 'introduced', 'became law'                                                                '''
    edge = getJSON('https://api.propublica.org/congress/v1/' + str(congress) + '/bills/' + bill_id + '/cosponsors.json')
    toReturn = False
    if edge['status'] == 'OK':
        if progress =='introduced':
            toReturn = True
        elif progress == 'became law':
            toReturn = edge['results'][0]['latest_major_action'][:17] == 'Became Public Law'           
                
    if toReturn:
        return (edge['results'][0]['sponsor_id'], [cosponsor['cosponsor_id'] for cosponsor in edge['results'][0]['cosponsors']])
    else:
        return ()
    
def getMembers(congress):
    '''takes congress session and Returns dictioanry of congress member ids with value 0'''
    senate_members = getJSON('https://api.propublica.org/congress/v1/'+ str(congress) +'/senate/members.json')
    house_members = getJSON('https://api.propublica.org/congress/v1/'+ str(congress) +'/house/members.json')
    senateList, houseList = [],[]
    if senate_members['status'] == 'OK':
        senateList = [(senator['id'],0) for senator in senate_members['results'][0]['members']]
    if house_members['status'] == 'OK':
        houseList = [(rep['id'],0) for rep in house_members['results'][0]['members']]
    return dict(senateList + houseList)

def getNameIDPairing(congress):
    senate_members = getJSON('https://api.propublica.org/congress/v1/'+ str(congress) +'/senate/members.json')
    house_members = getJSON('https://api.propublica.org/congress/v1/'+ str(congress) +'/house/members.json')
    senateList, houseList = [],[]
    if senate_members['status'] == 'OK':
        senateList = [(senator['id'], senator['first_name'] + ' ' + senator['last_name'] + '(' + senator['party'] + ')') for senator in senate_members['results'][0]['members']]
    if house_members['status'] == 'OK':
        houseList = [(rep['id'],rep['first_name'] + ' ' + rep['last_name'] + '(' + rep['party'] + ')') for rep in house_members['results'][0]['members']]
    return dict(senateList + houseList)
    
    
def buildGraph(members, congress):
    graph = {}
    for m in members:
        graph[m] = []
    
    for i in range(1, MOST_RECENT_HR_BILL+1):
        print("At HR{0}".format(i))
        edge = getSponsors("hr{0}".format(i), congress)
        if edge: # not (,)
            graph[edge[0]].extend(edge[1])
    
    for i in range(1, MOST_RECENT_S_BILL+1):
        print("At S{0}".format(i))
        edge = getSponsors("s{0}".format(i), congress)
        if edge: # not (,)
            graph[edge[0]].extend(edge[1])
        
    return graph

def buildFilteredGraph(members, congress):
    graph = {}
    for m in members:
        graph[m] = []
    
    for i in sorted(list(hr_passed_one_house)):
        print("At HR{0}".format(i))
        edge = getSponsors("hr{0}".format(i), congress)
        if edge: # not (,)
            graph[edge[0]].extend(edge[1])
    
    for i in sorted(list(senate_passed_one_house)):
        print("At S{0}".format(i))
        edge = getSponsors("s{0}".format(i), congress)
        if edge: # not (,)
            graph[edge[0]].extend(edge[1])
        
    return graph

def ranker(members, graph, p=.15, iterations=1000000):
    assert len(graph) == len(members)
    member_list = list(graph.keys())
    current = random.choice(member_list)
    members[current] += 1
    for i in range(iterations):
#         if i % 10 == 0:
#             print("Iteration: {0}".format(i))
        if len(graph[current]) == 0 or random.random() < p:
            current = random.choice(member_list)
#             print("reset")

        else:
            neighbors = graph[current]
            current = neighbors[random.randint(0, len(neighbors)-1)]
#             print("Continue")
        members[current] += 1
    return members


def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)


In [77]:
# members = getMembers(CONGRESS)
# graph = buildGraph(members, CONGRESS)
# names = getNameIDPairing(CONGRESS)
filteredGraph = buildFilteredGraph(members, CONGRESS)

save_obj(filteredGraph, 'congress' + str(CONGRESS) + '_filteredGraphAtLeastOneHouse')
# save_obj(members, 'congress' + str(CONGRESS) + '_members')
# save_obj(graph, 'congress' + str(CONGRESS) + '_graph')
# save_obj(names, 'congress' + str(CONGRESS) + '_names')

At HR2
At HR3
At HR5
At HR6
At HR7
At HR8
At HR10
At HR22
At HR23
At HR26
At HR30
At HR33
At HR34
At HR35
At HR36
At HR37
At HR50
At HR87
At HR91
At HR136
At HR158
At HR159
At HR160
At HR161
At HR172
At HR181
At HR185
At HR203
At HR208
At HR212
At HR216
At HR223
At HR233
At HR235
At HR237
At HR240
At HR246
At HR251
At HR280
At HR284
At HR285
At HR294
At HR295
At HR299
At HR313
At HR322
At HR323
At HR324
At HR329
At HR336
At HR348
At HR350
At HR351
At HR357
At HR360
At HR361
At HR373
At HR387
At HR398
At HR400
At HR404
At HR427
At HR431
At HR432
At HR433
At HR455
At HR460
At HR468
At HR469
At HR471
At HR474
At HR482
At HR487
At HR496
At HR511
At HR514
At HR515
At HR527
At HR529
At HR533
At HR538
At HR558
At HR596
At HR597
At HR598
At HR601
At HR606
At HR615
At HR622
At HR623
At HR636
At HR639
At HR644
At HR647
At HR648
At HR650
At HR651
At HR653
At HR670
At HR675
At HR677
At HR685
At HR692
At HR699
At HR702
At HR709
At HR710
At HR712
At HR719
At HR720
At HR723
At HR728
At HR734
At HR749

In [80]:
filteredGraph = load_obj('congress' + str(CONGRESS) + '_filteredGraphAtLeastOneHouse')
graph = load_obj('congress' + str(CONGRESS) + '_graph')
members = load_obj('congress' + str(CONGRESS) + '_members')
names = load_obj('congress' + str(CONGRESS) + '_names')

In [81]:
members = load_obj('congress' + str(CONGRESS) + '_members')
ranking = ranker(members, filteredGraph,.15, 100000)
member_ranks = [pair[0] for pair in sorted(members.items(), key = lambda item: item[1], reverse=True)]
top_score = members[member_ranks[0]]
for i in range(len(member_ranks)):
    print("%d %f: %s" % (i+1,members[member_ranks[i]]/top_score, names[member_ranks[i]]))

1 1.000000: Kyrsten Sinema(D)
2 0.879941: Lamar Smith(R)
3 0.717423: Derek Kilmer(D)
4 0.689605: Mark Kirk(R)
5 0.688141: Michael McCaul(R)
6 0.667643: Richard Blumenthal(D)
7 0.651537: Ron Johnson(R)
8 0.648609: Steve Stivers(R)
9 0.647145: Peter King(R)
10 0.629575: John Katko(R)
11 0.626647: Bennie Thompson(D)
12 0.603221: Kathleen Rice(D)
13 0.601757: Amata Radewagen(R)
14 0.597365: Martha McSally(R)
15 0.588580: Pete Sessions(R)
16 0.584187: Randy Hultgren(R)
17 0.551977: Tom Cole(R)
18 0.543192: Charles Schumer(D)
19 0.541728: Cedric Richmond(D)
20 0.540264: John Cornyn(R)
21 0.537335: David Schweikert(R)
22 0.530015: Candice Miller(R)
23 0.513909: Greg Walden(R)
24 0.510981: Patty Murray(D)
25 0.506589: Don Young(R)
26 0.505124: Norma Torres(D)
27 0.503660: Heidi Heitkamp(D)
28 0.499268: Gerald Connolly(D)
29 0.491947: Martin Heinrich(D)
30 0.487555: Luke Messer(R)
31 0.487555: Eddie Johnson(D)
32 0.486091: Richard Durbin(D)
33 0.486091: Chris Gibson(R)
34 0.481698: Christopher 

In [68]:
test = getJSON('https://api.propublica.org/congress/v1/members/N000147/bills/introduced.json')

In [139]:
test2 = getJSON('https://api.propublica.org/congress/v1/114/bills/hr6477/cosponsors.json')

In [135]:
test3 = getJSON('https://api.propublica.org/congress/v1/114/bills/hr6431/cosponsors.json')

In [136]:
test3 = getJSON('https://api.propublica.org/congress/v1/114/bills/hr6431/cosponsors.json')
test3['results'][0]['latest_major_action'][:17] == 'Became Public Law'

True

In [157]:
test4 = load_obj('congress' + str(CONGRESS) + '_filteredGraph')

In [72]:
print(len(senate_passed_one_house))
print(len(hr_passed_one_house))

195
773
